In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [8, 4.5]
plt.rcParams['figure.dpi'] = 300

# 텐서 씨드 고정
import tensorflow as tf
import random
import os
#글로벌 시드
class config:
    seed = 1234
    device = "cuda:0"    

def seed_everything(seed: int = 1234):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(config.seed)

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

seed =1234

pd.set_option('display.max_rows', 10, 'display.max_columns', 500, 'max_colwidth', None)

In [2]:
# df = pd.read_csv('./kospi.csv')
df = pd.read_csv('./baseline_hv.csv')

df.shape

(4598, 55)

In [3]:
df.head()

,date,baseline,HV,HV_pred,HV_resid,sum_word,MA5,MA20,MA60,MA120,MA5_gra,MA20_gra,MA60_gra,MA120_gra,V_MA5,V_MA20,V_MA60,V_MA120,EMA5,EMA20,EMA60,EMA120,MACD,MACD_sig9,MACD_OSC,Fast%K,Fast%D,Slow%K,Slow%D,RSI14,RSI signal,CCI,A_Ratio,B_Ratio,ATR,UpDI,DownDI,MFI,Momentum,Sonar,Sonar_sig5,STDEV,Sigma,TRIX,VR,WILLR,Psy,Disparity20,Disparity60,ADX,ADOSC,Volume_OSC,upperband,middleband,lowerband
0,2003-07-28,100.0,NaN,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2003-07-29,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2003-07-30,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2003-07-31,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2003-08-01,100.0,0.0,0.0,0.0,3.0,41.088806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.291079e+07,0.0,0.0,0.0,41.088806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.isna().sum()

date           0
baseline       0
HV             1
HV_pred       20
HV_resid      21
              ..
ADOSC          0
Volume_OSC     0
upperband      0
middleband     0
lowerband      0
Length: 55, dtype: int64

In [5]:
df.head()

,date,baseline,HV,HV_pred,HV_resid,sum_word,MA5,MA20,MA60,MA120,MA5_gra,MA20_gra,MA60_gra,MA120_gra,V_MA5,V_MA20,V_MA60,V_MA120,EMA5,EMA20,EMA60,EMA120,MACD,MACD_sig9,MACD_OSC,Fast%K,Fast%D,Slow%K,Slow%D,RSI14,RSI signal,CCI,A_Ratio,B_Ratio,ATR,UpDI,DownDI,MFI,Momentum,Sonar,Sonar_sig5,STDEV,Sigma,TRIX,VR,WILLR,Psy,Disparity20,Disparity60,ADX,ADOSC,Volume_OSC,upperband,middleband,lowerband
0,2003-07-28,100.0,NaN,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2003-07-29,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2003-07-30,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2003-07-31,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2003-08-01,100.0,0.0,0.0,0.0,3.0,41.088806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.291079e+07,0.0,0.0,0.0,41.088806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.set_index('date', inplace=True)

In [7]:
df.head()

,baseline,HV,HV_pred,HV_resid,sum_word,MA5,MA20,MA60,MA120,MA5_gra,MA20_gra,MA60_gra,MA120_gra,V_MA5,V_MA20,V_MA60,V_MA120,EMA5,EMA20,EMA60,EMA120,MACD,MACD_sig9,MACD_OSC,Fast%K,Fast%D,Slow%K,Slow%D,RSI14,RSI signal,CCI,A_Ratio,B_Ratio,ATR,UpDI,DownDI,MFI,Momentum,Sonar,Sonar_sig5,STDEV,Sigma,TRIX,VR,WILLR,Psy,Disparity20,Disparity60,ADX,ADOSC,Volume_OSC,upperband,middleband,lowerband
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-07-28,100.0,NaN,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003-07-29,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003-07-30,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003-07-31,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003-08-01,100.0,0.0,0.0,0.0,3.0,41.088806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.291079e+07,0.0,0.0,0.0,41.088806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df.shape

(4598, 54)

In [9]:
cols = ['HV', 'Sonar']#, 'MACD_OSC', 'Sonar', 'Sigma', 'Momentum', 'Volume_OSC',  'ATR','RSI signal','RSI14',  'ADOSC']
cols


['HV', 'Sonar']

In [10]:
df = df[cols]

In [11]:
df.index

Index(['2003-07-28', '2003-07-29', '2003-07-30', '2003-07-31', '2003-08-01',
       '2003-08-04', '2003-08-05', '2003-08-06', '2003-08-07', '2003-08-08',
       ...
       '2021-10-15', '2021-10-18', '2021-10-19', '2021-10-20', '2021-10-21',
       '2021-10-22', '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28'],
      dtype='object', name='date', length=4598)

In [12]:
df.head()

,HV,Sonar
date,,
2003-07-28,NaN,0.0
2003-07-29,0.0,0.0
2003-07-30,0.0,0.0
2003-07-31,0.0,0.0
2003-08-01,0.0,0.0


In [13]:
df.shape

(4598, 2)

In [14]:
feature_df = df.loc['2003-09-29':'2021-09-28', 'HV':'Sonar']
print(feature_df.shape)
feature_df.head()

(4532, 2)


,HV,Sonar
date,,
2003-09-29,0.000000,-0.468851
2003-09-30,0.046147,-0.717700
2003-10-01,0.111621,-0.808805
2003-10-02,0.113955,-0.772456
2003-10-03,0.127419,-0.492226


In [15]:
label_df = df.loc['2003-09-29':'2021-09-28', 'HV']
print(label_df.shape)
label_df.head()

(4532,)


date
2003-09-29    0.000000
2003-09-30    0.046147
2003-10-01    0.111621
2003-10-02    0.113955
2003-10-03    0.127419
Name: HV, dtype: float64

# TRAIN, TEST 분리
- 1차 : 2003 ~ 2011 학습 / 2012 ~ 2016 예측
- 2차 : 2003 ~ 2016 학습 / 2017~ 2021 예측

## 1차 train_test 분리

In [16]:
X_train_1st = feature_df.loc['2003-09-29':'2018-09-28']
print(X_train_1st.shape)
X_train_1st.head()

(3778, 2)


,HV,Sonar
date,,
2003-09-29,0.000000,-0.468851
2003-09-30,0.046147,-0.717700
2003-10-01,0.111621,-0.808805
2003-10-02,0.113955,-0.772456
2003-10-03,0.127419,-0.492226


In [17]:
y_train_1st = label_df.loc['2003-09-29':'2018-09-28']
print(y_train_1st.shape)
y_train_1st.head()

(3778,)


date
2003-09-29    0.000000
2003-09-30    0.046147
2003-10-01    0.111621
2003-10-02    0.113955
2003-10-03    0.127419
Name: HV, dtype: float64

### test

In [18]:
X_test_1st = feature_df.loc['2018-09-29':'2021-10-28']
print(X_test_1st.shape)
X_test_1st.head()

(754, 2)


,HV,Sonar
date,,
2018-10-01,0.083115,1.820228
2018-10-02,0.083981,1.737417
2018-10-03,0.078241,1.326049
2018-10-04,0.095435,0.621009
2018-10-05,0.095647,-0.080006


In [19]:
y_test_1st = label_df.loc['2018-09-29':'2021-10-28']
print(y_test_1st.shape)
y_test_1st.head()

(754,)


date
2018-10-01    0.083115
2018-10-02    0.083981
2018-10-03    0.078241
2018-10-04    0.095435
2018-10-05    0.095647
Name: HV, dtype: float64

In [20]:
print(X_train_1st.shape, y_train_1st.shape)
print(X_test_1st.shape, y_test_1st.shape)

(3778, 2) (3778,)
(754, 2) (754,)


## scaling
X와 y를 각각 scaling 해주어야한다.

In [21]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Standardization 평균 0 / 분산 1
X_scaler = MinMaxScaler()

# 교차검증시
X_scaler.fit(X_train_1st)
X_train_1st_sc = X_scaler.transform(X_train_1st)
X_test_1st_sc = X_scaler.transform(X_test_1st)

In [22]:
# from sklearn.preprocessing import StandardScaler

# Standardization 평균 0 / 분산 1
y_scaler = MinMaxScaler()   

# 교차검증시
y_scaler.fit(np.array(y_train_1st).reshape(-1,1))
y_train_1st_sc = y_scaler.transform(np.array(y_train_1st).reshape(-1,1))
y_test_1st_sc = y_scaler.transform(np.array(y_test_1st).reshape(-1,1))

In [23]:
X_train_1st_sc.shape, X_test_1st_sc.shape, y_train_1st.shape, y_test_1st.shape

((3778, 2), (754, 2), (3778,), (754,))

In [24]:
y_train_1st_sc[:5]

array([[0.        ],
       [0.04309432],
       [0.1042375 ],
       [0.10641735],
       [0.11899137]])

In [25]:
y_test_1st_sc[:5]

array([[0.07761774],
       [0.07842568],
       [0.0730659 ],
       [0.08912282],
       [0.08932082]])

In [26]:
y_train_1st[:5]

date
2003-09-29    0.000000
2003-09-30    0.046147
2003-10-01    0.111621
2003-10-02    0.113955
2003-10-03    0.127419
Name: HV, dtype: float64

In [27]:
y_test_1st[:5]

date
2018-10-01    0.083115
2018-10-02    0.083981
2018-10-03    0.078241
2018-10-04    0.095435
2018-10-05    0.095647
Name: HV, dtype: float64

In [28]:
# 입력 파라미터 feature, label -> numpy type
def make_sequence_dataset(feature, label, window_size):
    feature_list = [] # 생성될 feature list
    label_list = []  # 생성될 label list
    
    for i in range(len(feature)-window_size):
        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])
    return np.array(feature_list), np.array(label_list) # 3차원으로 만들어줌

In [29]:
window_size = 20
X_train_1st, y_train_1st = make_sequence_dataset(X_train_1st_sc, y_train_1st_sc, window_size)
X_test_1st, y_test_1st = make_sequence_dataset(X_test_1st_sc, y_test_1st_sc, window_size)
print(X_train_1st.shape, y_train_1st.shape, X_test_1st.shape, y_test_1st.shape) # (batch size, time steps, input dims)

(3758, 20, 2) (3758, 1) (734, 20, 2) (734, 1)


# TRAIN, TEST 분리

In [30]:
# input_shape
X_train_1st[0].shape

(20, 2)

# Model

In [31]:
# Initialising the RNN
model_118 = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
model_118.add(LSTM(units = 10, activation = 'relu', return_sequences = True, input_shape = X_train_1st[0].shape))
model_118.add(Dropout(0.1))

# Adding a second LSTM layer and some Dropout regularisation
model_118.add(LSTM(units = 10, activation = 'relu', return_sequences = True))
model_118.add(Dropout(0.1))

# Adding a third LSTM layer and some Dropout regularisation
model_118.add(LSTM(units = 10, activation = 'relu', return_sequences = True))
model_118.add(Dropout(0.1))

# Adding a fourth LSTM layer and some Dropout regularisation
model_118.add(LSTM(units = 10, activation = 'relu'))
model_118.add(Dropout(0.1))

# Adding the output layer
model_118.add(Dense(units = 1))
model_118.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 10)            520       
_________________________________________________________________
dropout (Dropout)            (None, 20, 10)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 10)            840       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 10)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 10)            840       
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 10)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10)                8

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

# 출처: https://kibua20.tistory.com/206 [모바일 SW 개발자가 운영하는 블로그]
sgd = optimizers.SGD(learning_rate = 0.0001)
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_118.compile(loss = 'mse', optimizer = 'adam')
early_stop = EarlyStopping(monitor = 'val_loss', patience = 10)
history = model_118.fit(X_train_1st, y_train_1st, validation_data = (X_test_1st, y_test_1st), epochs=100, 
          batch_size=15, callbacks = [early_stop])

Epoch 1/100
251/251 [==============================] - 7s 15ms/step - loss: 0.0101 - val_loss: 0.0131
Epoch 2/100
251/251 [==============================] - 5s 20ms/step - loss: 0.0041 - val_loss: 0.0110
Epoch 3/100
251/251 [==============================] - 4s 14ms/step - loss: 0.0033 - val_loss: 0.0080
Epoch 4/100
251/251 [==============================] - 5s 20ms/step - loss: 0.0028 - val_loss: 0.0063
Epoch 5/100
251/251 [==============================] - 4s 14ms/step - loss: 0.0024 - val_loss: 0.0053
Epoch 6/100
251/251 [==============================] - 5s 20ms/step - loss: 0.0024 - val_loss: 0.0045
Epoch 7/100
251/251 [==============================] - 4s 14ms/step - loss: 0.0021 - val_loss: 0.0041
Epoch 8/100
251/251 [==============================] - 5s 20ms/step - loss: 0.0020 - val_loss: 0.0032
Epoch 9/100
251/251 [==============================] - 4s 14ms/step - loss: 0.0018 - val_loss: 0.0034
Epoch 10/100
251/251 [==============================] - 5s 20ms/step - loss: 0.001

In [ ]:
val_loss = history.history['val_loss']
print(val_loss[:5])

In [ ]:
train_loss = history.history['loss']
print(train_loss[:5])

In [ ]:
plt.plot(np.arange(len(val_loss)), val_loss, c="red", label='val')
plt.plot(np.arange(len(train_loss)), train_loss, c='blue', label ='train')
plt.legend()
plt.show()

In [ ]:
rmse_score = np.sqrt(np.min(val_loss))
rmse_score

# 예측 결과 시각화
- https://www.youtube.com/watch?v=LLLVYkXJw30

In [ ]:
pred = model_118.predict(X_test_1st)

In [ ]:
pred[:5]

In [ ]:
np.mean(pred)

In [ ]:
y_test_1st[:5]

In [ ]:
plt.figure(figsize = (12,6))
plt.title('STDEV + STDEV_log + HV, window_size = 20')
plt.ylabel('HV')
plt.xlabel('Period')
plt.plot(y_test_1st, label = 'actual')
plt.plot(pred, label = 'prediction')
plt.grid()
plt.legend(loc = 'best')

plt.show()